+ You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships.
+ They need your help them give accurate estimates of how many crew members a ship will require
+ They are currently selling ships to some new customers and want you to create a model and use it to predict how many crew members the ships will need.
+ Your job is to create a regression model that will help predict how many crew members will be needed for future ships.
+ The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis.

In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')

from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('cruise').getOrCreate()

In [2]:
df = spark.read.csv('cruise_ship_info.csv',
                      inferSchema=True,
                      header=True)

In [3]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [5]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [6]:
from pyspark.ml.feature import StringIndexer

In [7]:
indexer = StringIndexer(
    inputCol='Cruise_line',
    outputCol='cruise_cat'
)
indexed = indexer.fit(df).transform(df)
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [10]:
assembler = VectorAssembler(
    inputCols=['Age',
               'Tonnage',
               'passengers',
               'length',
               'cabins',
               'passenger_density',
               'cruise_cat'],
    outputCol='features'
)

In [11]:
output = assembler.transform(indexed)

In [12]:
output.select('features', 'crew').show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [13]:
final_data = output.select(['features', 'crew'])

In [14]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml.regression import LinearRegression

In [16]:
ship_lr = LinearRegression(labelCol='crew')

In [17]:
trained_ship_model = ship_lr.fit(train_data)

In [18]:
ship_results = trained_ship_model.evaluate(test_data)

In [19]:
ship_results.rootMeanSquaredError

1.3674179117447047

In [20]:
ship_results.r2

0.8436443568485994

In [21]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              119|
|   mean| 7.73546218487396|
| stddev|3.516338317899853|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [22]:
ship_results.meanAbsoluteError

0.8026099880748107

In [23]:
from pyspark.sql.functions import corr

In [24]:
df.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [25]:
df.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

